# From Detection to Credibility: A Machine Learning Framework for Assessing News Source Reliability



**Motivation**


### Fake News Classification Dataset Information

The [Fake News Classification Dataset](https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification) contains real and fake news articles.

- **Index:**: Index.
- **title:** Title of news article.
- **text:** Text content of news article.
- **label:** Whether news article is real (1) or fake (0).

The Fake News Dataset is split into 3 `csv` files (`part1.csv`, `part2.csv`, `part3.csv`) so that size does not exceed size limit to push changes to GitHub.

## Import Libraries

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Text Preprocessing and NLP
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud


## Data Preparation (Loading CSV)

Load the 3 Fake News `csv` files into pandas DataFrames
- `part1.csv` is loaded into `data_raw1` DataFrame.
- `part2.csv` is loaded into `data_raw2` DataFrame.
- `part3.csv` is loaded into`data_raw3` DataFrame.

In [13]:
data_raw1 = pd.read_csv('part1.csv')
data_raw2 = pd.read_csv('part2.csv')
data_raw3 = pd.read_csv('part3.csv')

In [15]:
data_raw1.info()
print("Dataframe 1 Shape: ", data_raw1.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24044 entries, 0 to 24043
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24044 non-null  int64 
 1   title       23894 non-null  object
 2   text        24033 non-null  object
 3   label       24044 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 751.5+ KB
Dataframe 1 Shape:  (24044, 4)


In [16]:
data_raw2.info()
print("Dataframe 2 Shape: ", data_raw2.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24044 entries, 0 to 24043
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24044 non-null  int64 
 1   title       23849 non-null  object
 2   text        24030 non-null  object
 3   label       24044 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 751.5+ KB
Dataframe 2 Shape:  (24044, 4)


In [17]:
data_raw3.info()
print("Dataframe 3 Shape: ", data_raw3.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24046 entries, 0 to 24045
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24046 non-null  int64 
 1   title       23833 non-null  object
 2   text        24032 non-null  object
 3   label       24046 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 751.6+ KB
Dataframe 3 Shape:  (24046, 4)


In [20]:
data_raw1.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [21]:
data_raw2.head()

,Unnamed: 0,title,text,label
0,24044,Massachusetts governor would sign transgender ...,BOSTON (Reuters) - Massachusetts Governor Char...,0
1,24045,"Weakened Militarily, ISIS Still Has Power to S...","In the past few weeks, the Islamic State has s...",0
2,24046,SARAH JESSICA PARKER FEARS She’ll Be Attacked ...,"With absolutely NOTHING to back it up, Sarah J...",1
3,24047,Rosenstein may need to recuse himself from Rus...,WASHINGTON (Reuters) - U.S. Deputy Attorney Ge...,0
4,24048,Cop Arrested for Using Police Status to Rape L...,Home / Badge Abuse / Cop Arrested for Using Po...,1


In [22]:
data_raw3.head()

,Unnamed: 0,title,text,label
0,48088,Blue State Blues: The White House Loves the Ru...,The Trump White House seems quietly to be enjo...,0
1,48089,"Harassment Crisis Builds at Fox News, Despite ...",When the anchor Gretchen Carlson filed a bombs...,0
2,48090,Former Ku Klux Klan leader Duke runs for U.S. ...,"BATON ROUGE, La. (Reuters) - David Duke, a for...",0
3,48091,NYTIMES: Sessions’ and Bannon’s Department of ...,Emily Bazelon writes in the New York Times on ...,0
4,48092,UPDATE: WIKILEAKS CLINTON Document Dump Back On!,Hillary Clinton strategist Bob Beckel called f...,1


## Combine all 3 dataframes into 1
Here we combine all 3 dataframes (`data_raw1`, `data_raw2`, `data_raw3`) into 1 dataframe (`data_raw`) by concatenating along rows.

We also reset the `index` and drops the old `index` column

In [37]:
# Combining all 3 dataframes into 1
data = pd.concat([data_raw1, data_raw2, data_raw3], axis=0)

# Reset index and drop old index column
data = data.reset_index(drop=True)

data.info()
print("Dataframe Shape: ", data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB
Dataframe Shape:  (72134, 4)


In [39]:
data

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
...,...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,0
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",1
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,0
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,0
